In [1]:
"""Contains functions to compute soil freezing/thawing depending
on soil temperatures, moisture and heat flux of a soil profile. 
Contains:
- init_temp_vars() # initialize temporary help variables
- compute_soil_freezing() # main program

"""
import numpy as np
import soil_constants as sc

In [12]:
def init_temp_vars(nsoil):
    return np.zeros(nsoil)
def compute_soil_freezing(nsoil=5,
                          soil_thick_sl=np.full(5,fill_value=0.05),
                          soil_bulk_dens_sl=np.full(5,fill_value=1),
                          soil_heat_capa_sl= np.full(5,fill_value=1000),
                          soil_w_fc_sl = np.full(5,fill_value=1.2),
                          soil_t_sl=np.full(5,fill_value=293.15),
                          soil_w_sl=np.ones(5),
                          soil_icew_sl=np.ones(5),
                         soil_heat_sl=np.zeros(5)):
    
    """Compute soil freezing based on soil profile parameters, soil temperature, moisture
    and layer heat fluxes
    Inputs:
    - nsoil : number of soil layers [-]
    - soil_thick_sl : soil thickness [m]
    - soil_heat_capacity_dens_sl [kg m-1]
    - soil_bulk_dens_sl [kg m-1]
    - soil_w_capa_sl : volumetric soil water field capacity for each layer layer [m]
    - soil_t_sl : soil temperature per layer [K]
    - soil_w_sl : volumetric soil water per layer [m]
    - soil_icew_sl : volumetric ice water per layer in water equivalent [m]
    - soil_heat_sl : heat transfer per layer (downwards) [kJ m-2 s-1]

    """
    ###Soil Freezing###
    # condition for frozen temperature and available liquid water in soil layers 
    frozen_iswater_condition = np.where((soil_t_sl[:] < sc.Tzero) & (soil_w_sl[:] > 0))


    # Calculate heat deficit past freezing point 
    hlp1 = init_temp_vars(nsoil)
    hlp1[frozen_iswater_condition] = soil_heat_capa_sl[frozen_iswater_condition]  \
                                    * soil_bulk_dens_sl[frozen_iswater_condition] \
                                    * soil_thick_sl[frozen_iswater_condition] \
                                    * (soil_t_sl[frozen_iswater_condition] - sc.Tzero) 
    # Maximum latent heat release from freezing per soil layer
    hlp2 = init_temp_vars(nsoil)
    hlp2[frozen_iswater_condition] = sc.latent_heat_fusion * sc.w_dens * soil_w_sl[frozen_iswater_condition]
    
    # Define heat deficit condition
    # heat deficit equal or less than total potential release of fusion from soil layer
    heatdef_le_fusheat_condition = np.where(-hlp2 <= hlp1) 
    # heat deficit is greater than total potential release of fusion from soil layer
    heatdef_gt_fusheat_condition = np.where(-hlp2 > hlp1)
    w_soil_freeze_flx = init_temp_vars(nsoil)
    w_soil_freeze_flx[heatdef_gt_fusheat_condition] = np.minimum(soil_w_sl[heatdef_gt_fusheat_condition],
                                                    - hlp1[heatdef_gt_fusheat_condition] \
                                                    / (sc.w_dens * sc.latent_heat_fusion))
    w_soil_freeze_flx[heatdef_gt_fusheat_condition] = np.minimum(w_soil_freeze_flx[heatdef_gt_fusheat_condition] \
                                                    * sc.rel_vol_w2ice,
                                                    soil_w_fc_sl[heatdef_gt_fusheat_condition] \
                                                    - soil_icew_sl[heatdef_gt_fusheat_condition] * 
                                                    sc.rel_vol_w2ice)
    return soil_t_sl,soil_w_sl,soil_icew_sl

In [13]:
#Test
soil_t_sl,soil_w_sl,soil_ice_sl = compute_soil_freezing()
#Give output
print("soil layer temperatures [K]",soil_t_sl)
print("soil layer water [m]", soil_w_sl)
print("soil_ice_sl",soil_ice_sl)

soil layer temperatures [K] [293.15 293.15 293.15 293.15 293.15]
soil layer water [m] [1. 1. 1. 1. 1.]
soil_ice_sl [1. 1. 1. 1. 1.]


IndentationError: unexpected indent (<ipython-input-47-827c23714b74>, line 2)